In [1]:
import math
import numpy as np
import random
from copy import deepcopy

In [2]:
class Connect4:
    def __init__(self, rows=6, cols=7, connect=4):
        self.rows = rows
        self.cols = cols
        self.connect = connect
        self.reset()

    def reset(self):
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.current_player = 1  # 1 = X, -1 = O
        self.last_move = None
        self.done = False
        self.winner = None
        return self.board.copy()

    def valid_actions(self):
        return [c for c in range(self.cols) if self.board[0, c] == 0]

    def step(self, action):
        if action not in self.valid_actions():
            raise ValueError("Invalid action")

        # drop piece
        for r in range(self.rows - 1, -1, -1):
            if self.board[r, action] == 0:
                self.board[r, action] = self.current_player
                self.last_move = (r, action)
                break

        winner = check_winner(self.board, self.connect, self.last_move)
        if winner is not None:
            self.done = True
            self.winner = winner
            reward = 1 if winner == self.current_player else -1
        elif np.all(self.board != 0):
            self.done = True
            self.winner = 0
            reward = 0
        else:
            reward = 0

        if not self.done:
            self.current_player *= -1

        return self.board.copy(), reward, self.done, {}

    def render(self):
        sym = {1: 'X', -1: 'O', 0: '.'}
        for r in range(self.rows):
            print(" ".join(sym[int(x)] for x in self.board[r]))
        print(" ".join(str(i) for i in range(self.cols)))
        print()

In [3]:
def check_winner(board, connect=4, last_move=None):
    rows, cols = board.shape

    if last_move is not None:
        r0, c0 = last_move
        player = board[r0, c0]
        if player == 0:
            return None

        dirs = [(1,0), (0,1), (1,1), (-1,1)]
        for dr, dc in dirs:
            count = 1

            rr, cc = r0 + dr, c0 + dc
            while 0 <= rr < rows and 0 <= cc < cols and board[rr,cc] == player:
                count += 1
                rr += dr; cc += dc

            rr, cc = r0 - dr, c0 - dc
            while 0 <= rr < rows and 0 <= cc < cols and board[rr,cc] == player:
                count += 1
                rr -= dr; cc -= dc

            if count >= connect:
                return player

        return None

    # fallback: full-board scan
    for r in range(rows):
        for c in range(cols):
            if board[r,c] == 0:
                continue
            player = board[r,c]
            for dr, dc in [(1,0),(0,1),(1,1),(-1,1)]:
                cnt = 0
                rr, cc = r, c
                while 0 <= rr < rows and 0 <= cc < cols and board[rr,cc] == player:
                    cnt += 1
                    rr += dr; cc += dc
                if cnt >= connect:
                    return player
    return None

In [4]:
def heuristic_evaluate(board, player):
    rows, cols = board.shape
    opponent = -player

    SCORE = {4: 100000, 3: 100, 2: 10}
    total = 0

    def score_window(window):
        s = 0
        cnt_p = np.count_nonzero(window == player)
        cnt_o = np.count_nonzero(window == opponent)
        cnt_e = np.count_nonzero(window == 0)

        if cnt_p == 4:
            s += SCORE[4]
        elif cnt_p == 3 and cnt_e == 1:
            s += SCORE[3]
        elif cnt_p == 2 and cnt_e == 2:
            s += SCORE[2]

        if cnt_o == 4:
            s -= SCORE[4]
        elif cnt_o == 3 and cnt_e == 1:
            s -= SCORE[3]*0.9
        elif cnt_o == 2 and cnt_e == 2:
            s -= SCORE[2]*0.5

        return s

    # horizontal
    for r in range(rows):
        for c in range(cols-3):
            window = board[r, c:c+4]
            total += score_window(window)

    # vertical
    for c in range(cols):
        for r in range(rows-3):
            window = board[r:r+4, c]
            total += score_window(window)

    # diag down-right
    for r in range(rows-3):
        for c in range(cols-3):
            window = np.array([board[r+i, c+i] for i in range(4)])
            total += score_window(window)

    # diag up-right
    for r in range(3, rows):
        for c in range(cols-3):
            window = np.array([board[r-i, c+i] for i in range(4)])
            total += score_window(window)

    # center column preference
    center = cols // 2
    total += np.count_nonzero(board[:, center] == player) * 3

    return total

In [5]:
def minimax_ab(board, depth, alpha, beta, maximizing, player_to_move, env):
    w = check_winner(board, env.connect, last_move=None)
    if w is not None:
        return (1e9 if w == 1 else -1e9), None
    if np.all(board != 0):
        return 0, None
    if depth == 0:
        return heuristic_evaluate(board, player_to_move), None

    valid_moves = [c for c in range(env.cols) if board[0, c] == 0]
    valid_moves.sort(key=lambda c: -abs(c - env.cols//2))  # center-first move ordering

    best_move = None

    if maximizing:
        value = -math.inf
        for col in valid_moves:
            new_board = board.copy()
            for r in range(env.rows-1, -1, -1):
                if new_board[r, col] == 0:
                    new_board[r, col] = player_to_move
                    last_r = r
                    break

            w = check_winner(new_board, env.connect, last_move=(last_r, col))
            if w is not None:
                v = 1e9
            else:
                v, _ = minimax_ab(new_board, depth-1, alpha, beta, False, -player_to_move, env)

            if v > value:
                value = v
                best_move = col

            alpha = max(alpha, value)
            if alpha >= beta:
                break

        return value, best_move

    else:
        value = math.inf
        for col in valid_moves:
            new_board = board.copy()
            for r in range(env.rows-1, -1, -1):
                if new_board[r, col] == 0:
                    new_board[r, col] = player_to_move
                    last_r = r
                    break

            w = check_winner(new_board, env.connect, last_move=(last_r, col))
            if w is not None:
                v = -1e9
            else:
                v, _ = minimax_ab(new_board, depth-1, alpha, beta, True, -player_to_move, env)

            if v < value:
                value = v
                best_move = col

            beta = min(beta, value)
            if alpha >= beta:
                break

        return value, best_move


In [6]:
def minimax_agent(env, depth=4):
    board = env.board.copy()
    player = env.current_player
    value, action = minimax_ab(board, depth, -math.inf, math.inf, True, player, env)
    if action is None:
        return random.choice(env.valid_actions())
    return action

In [8]:
# -------------------------
# Full game loop (Human vs Minimax AI)
# -------------------------
env = Connect4()
game_over = False

print("Starting Connect Four")
env.render()

# 0 = Human, 1 = AI
turn = 0

while not game_over:

    if turn == 0:
        # Human move
        valid = env.valid_actions()
        col = None
        while col not in valid:
            try:
                col_input = int(input(f"Your move {valid}: "))
                if col_input in valid:
                    col = col_input
                else:
                    print("Invalid column. Choose from", valid)
            except:
                print("Enter a valid integer column.")
        
        _, _, game_over, _ = env.step(col)

        if env.winner == 1:
            env.render()
            print("You win!")
            break

    else:
        # AI move using minimax
        print("AI thinking...")
        col = minimax_agent(env, depth=4)
        _, _, game_over, _ = env.step(col)

        if env.winner == -1:
            env.render()
            print("AI wins!")
            break

    env.render()
    turn = 1 - turn  # switch turns

    # Check draw
    if game_over and env.winner == 0:
        print("Draw!")
        break

Starting Connect Four
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
0 1 2 3 4 5 6

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
0 1 2 3 4 5 6

AI thinking...
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . O . . .
0 1 2 3 4 5 6

. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . X O . . .
0 1 2 3 4 5 6

AI thinking...
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
X . X O . . .
0 1 2 3 4 5 6

. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . . O . . .
X . X O . . .
0 1 2 3 4 5 6

AI thinking...
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . O O . . .
X . X O . . .
0 1 2 3 4 5 6

. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
X . O O . . .
X . X O . . .
0 1 2 3 4 5 6

AI thinking...
. . . . . . .
. . . . . . .
. . . . . . .
O . . X . . .
X . O O . . .
X . X O . . .
0 1 2 3 4 5 6

. . . . . . .
. . . . . . .